In [23]:
import re
import random
import string
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from tqdm import tqdm

# Limpiar datos

In [ ]:
# Funciones de limpieza de datos para los datos en inglés y en español

def clean_text(text):
    """
    Limpia el texto en inglés eliminando enlaces, normalizando a minúsculas, 
    eliminando caracteres especiales y números (excepto comas y puntos), 
    reduciendo múltiples espacios en blanco y eliminando saltos de línea.
    """
    text = re.sub(r'https?\s*:\s*//\s*\S+', '', text)
    text = re.sub(r'www\.?\s*\S+', '', text)
    text = re.sub(r'http[s]?://\S+', '', text)    
    text = text.lower()   
    # Mantener comas y puntos)
    text = re.sub(r'[^a-z\s,.]', '', text)  
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\n', '', text)
    return text



def clean_text_es(text):
    """
    Limpia el texto en español eliminando enlaces, normalizando a minúsculas, 
    eliminando caracteres especiales y números (excepto caracteres acentuados, comas y puntos), 
    reduciendo múltiples espacios en blanco y eliminando saltos de línea.
    """
    text = re.sub(r'https?\s*:\s*//\s*\S+', '', text)
    text = re.sub(r'www\.?\s*\S+', '', text)
    text = re.sub(r'http[s]?://\S+', '', text)
    text = text.lower()   
    text = re.sub(r'[^a-záéíóúñü\s,.]', '', text)  
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\n', '', text)

    return text

## Aplicar a datos de train y test

In [1]:

df = pd.read_json(r"data/train/dataset_en_train.json")
df['text'] = df['text'].apply(lambda text: clean_text_en(text))

df = pd.read_json(r"data/train/dataset_es_train.json")
df['text'] = df['text'].apply(lambda text: clean_text_es(text))

df = pd.read_json(r"data/test/dataset_en_test.json")
df['text'] = df['text'].apply(lambda text: clean_text_en(text))

df = pd.read_json(r"data/test/dataset_es_test.json")
df['text'] = df['text'].apply(lambda text: clean_text_es(text))

#df.to_csv(r"data/train/dataset_en_train_cleaned.csv", index=False)
#df.to_csv(r"data/test/dataset_en_test_cleaned.csv", index=False)
#df.to_csv(r"data/train/dataset_es_train_cleaned.csv", index=False)
#df.to_csv(r"data/test/dataset_es_test_cleaned.csv", index=False)

'\ndf.to_csv(r"data/train/dataset_es_train_cleaned.csv", index=False)\n\ndf = pd.read_json(r"data/train/dataset_en_train.json")\ndf[\'text\'] = df[\'text\'].apply(lambda text: clean_text(text))\ndf.head()\ndf.to_csv(r"data/train/dataset_en_train_cleaned.csv", index=False)'

# Aumento de Datos con back-translation

## Aumento datos de entrenamiento español

In [ ]:
# Cargar el DataFrame

df=pd.read_csv('data/train/dataset_es_train_cleaned.csv')

In [ ]:
# Cargar los modelos y tokenizadores de traducción
model_name_es_en = "Helsinki-NLP/opus-mt-es-en"
model_name_en_es = "Helsinki-NLP/opus-mt-en-es"

tokenizer_es_en = MarianTokenizer.from_pretrained(model_name_es_en)
model_es_en = MarianMTModel.from_pretrained(model_name_es_en)

tokenizer_en_es = MarianTokenizer.from_pretrained(model_name_en_es)
model_en_es = MarianMTModel.from_pretrained(model_name_en_es)

def translate_text(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    translated = model.generate(**inputs)
    return tokenizer.batch_decode(translated, skip_special_tokens=True)[0]

new_rows = []
for i, fila in tqdm(df.iterrows(), total=df.shape[0], desc="Procesando filas"):
    # Traducir de español a inglés
    text_en = translate_text(fila['text'], tokenizer_es_en, model_es_en)
    # Traducir de vuelta de inglés a español
    text_es_back = translate_text(text_en, tokenizer_en_es, model_en_es)

    # Generar un nuevo ID aleatorio para nuevos datos y mantener originales
    id_new = ''.join(random.choices(string.ascii_letters + string.digits, k=5))
    new_row = {'id': id_new, 'text': text_es_back, 'category': fila['category']}
    new_rows.append(new_row)

new_df = pd.DataFrame(new_rows)
# Concatenar el DataFrame original con el nuevo DataFrame
df = pd.concat([df, new_df], ignore_index=True)

# Guardar el DataFrame resultante 
df.to_csv('data/train/dataset_es_train_augmented.csv', index=False)


/opt/conda/lib/python3.8/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Procesando filas:  12%|█▏        | 471/4000 [46:49<5:35:25,  5.70s/it] 

## Aumento de datos inglés

In [ ]:
# Cargar el DataFrame
df = pd.read_csv("data/train/dataset_en_train_cleaned.csv")

In [ ]:
# Cargar los modelos y tokenizadores de traducción
model_name_es_en = "Helsinki-NLP/opus-mt-en-de"
model_name_en_es = "Helsinki-NLP/opus-mt-de-en"

tokenizer_es_en = MarianTokenizer.from_pretrained(model_name_es_en)
model_es_en = MarianMTModel.from_pretrained(model_name_es_en)

tokenizer_en_es = MarianTokenizer.from_pretrained(model_name_en_es)
model_en_es = MarianMTModel.from_pretrained(model_name_en_es)

def translate_text(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    translated = model.generate(**inputs)
    return tokenizer.batch_decode(translated, skip_special_tokens=True)[0]

new_rows = []
for i, fila in tqdm(df.iterrows(), total=df.shape[0], desc="Procesando filas"):
    # Traducir de inglés a alemán
    text_en = translate_text(fila['text'], tokenizer_es_en, model_es_en)
    # Traducir de vuelta de alemán a inglés
    text_es_back = translate_text(text_en, tokenizer_en_es, model_en_es)
    id_new = ''.join(random.choices(string.ascii_letters + string.digits, k=5))
    new_row = {'id': id_new, 'text': text_es_back, 'category': fila['category']}
    new_rows.append(new_row)
new_df = pd.DataFrame(new_rows)

# Concatenar el DataFrame original con el nuevo DataFrame
df = pd.concat([df, new_df], ignore_index=True)

# Guardar el DataFrame resultante como augmented
df.to_csv('data/train/dataset_en_train_augmented.csv', index=False)
